# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


nhl_df = nhl_df[nhl_df.year == 2018]
nhl_df = nhl_df[nhl_df.columns[:4]]
nhl_df.team.replace(to_replace='\*', value='', regex=True, inplace=True)
nhl_df.replace({'Tampa Bay Lightning': 'Lightning',
                 'Boston Bruins': 'Bruins',
                 'Toronto Maple Leafs': 'Maple Leafs',
                 'Florida Panthers': 'Panthers',
                 'Detroit Red Wings': 'Red Wings',
                 'Montreal Canadiens': 'Canadiens',
                 'Ottawa Senators': 'Senators',
                 'Buffalo Sabres': 'Sabres',
                 'Washington Capitals': 'Capitals',
                 'Pittsburgh Penguins': 'Penguins',
                 'Philadelphia Flyers': 'Flyers',
                 'Columbus Blue Jackets': 'Blue Jackets',
                 'New Jersey Devils': 'Devils',
                 'Carolina Hurricanes': 'Hurricanes',
                 'New York Islanders': 'Islanders',
                 'New York Rangers': 'Rangers',
                 'Nashville Predators': 'Predators',
                 'Winnipeg Jets': 'Jets',
                 'Minnesota Wild': 'Wild',
                 'Colorado Avalanche': 'Avalanche',
                 'St. Louis Blues': 'Blues',
                 'Dallas Stars': 'Stars',
                 'Chicago Blackhawks': 'Blackhawks',
                 'Vegas Golden Knights': 'Golden Knights',
                 'Anaheim Ducks': 'Ducks',
                 'San Jose Sharks': 'Sharks',
                 'Los Angeles Kings': 'Kings',
                 'Calgary Flames': 'Flames',
                 'Edmonton Oilers': 'Oilers',
                 'Vancouver Canucks': 'Canucks',
                 'Arizona Coyotes': 'Coyotes'
                 }, inplace=True)


cities = cities[cities.columns[[0, 1, -1]]]
cities = cities.replace(to_replace='\[(.*?)\]', value='', regex=True)
cities = cities.replace({'—': np.NaN, '': np.NaN}).dropna()
cities.rename(columns={cities.columns[1]: 'population', cities.columns[0]: 'area'}, inplace=True)
cities.loc[0].NHL = ['Rangers', 'Islanders', 'Devils']
cities.loc[1].NHL = ['Kings', 'Ducks']
cities = cities.explode('NHL').reset_index(drop=True)
cities.rename(columns={'NHL': 'team'}, inplace=True)

df = pd.merge(nhl_df, cities, on='team')
columns = ['GP', 'W', 'L', 'population']
for col in columns:
    df[col] = pd.to_numeric(df[col])
df['win_rate'] = df.W / (df.W + df.L)

def nhl_correlation(): 
    global data_nhl
    data_nhl = df.groupby('area').mean()[['population', 'win_rate']]
    
    
    population_by_region = [i for i in data_nhl.population] # pass in metropolitan area population from cities
    win_loss_by_region = [i for i in data_nhl.win_rate] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

nba_df = nba_df[nba_df.year==2018]
nba_df = nba_df[nba_df.columns[:4]]
nba_df.replace(to_replace='\*?\s\(\d+\)', value='', regex=True, inplace=True)
nba_df.rename(columns={'W/L%': 'win_rate'}, inplace=True)
nba_df['team'].replace({'Toronto Raptors': 'Raptors',
                        'Boston Celtics': 'Celtics',
                        'Philadelphia 76ers': '76ers',
                        'Cleveland Cavaliers': 'Cavaliers',
                        'Indiana Pacers': 'Pacers',
                        'Miami Heat': 'Heat',
                        'Milwaukee Bucks': 'Bucks',
                        'Washington Wizards': 'Wizards',
                        'Detroit Pistons': 'Pistons',
                        'Charlotte Hornets': 'Hornets',
                        'New York Knicks': 'Knicks',
                        'Brooklyn Nets': 'Nets',
                        'Chicago Bulls': 'Bulls',
                        'Orlando Magic': 'Magic',
                        'Atlanta Hawks': 'Hawks',
                        'Houston Rockets': 'Rockets',
                        'Golden State Warriors': 'Warriors',
                        'Portland Trail Blazers': 'Trail Blazers',
                        'Oklahoma City Thunder': 'Thunder',
                        'Utah Jazz': 'Jazz',
                        'New Orleans Pelicans': 'Pelicans',
                        'San Antonio Spurs': 'Spurs',
                        'Minnesota Timberwolves': 'Timberwolves',
                        'Denver Nuggets': 'Nuggets',
                        'Los Angeles Clippers': 'Clippers',
                        'Los Angeles Lakers': 'Lakers',
                        'Sacramento Kings': 'Kings',
                        'Dallas Mavericks': 'Mavericks',
                        'Memphis Grizzlies': 'Grizzlies',
                        'Phoenix Suns': 'Suns'}, inplace=True)

cities = cities.iloc[:, [0, 1, -2]]
cities = cities.replace(to_replace='\[(.*?)\]', value='', regex=True)
cities = cities.replace({'—': np.NaN, '': np.NaN}).dropna()
cities.rename(columns={cities.columns[1]: 'population', cities.columns[0]: 'area'}, inplace=True)
cities.loc[0].NBA = ['Knicks', 'Nets']
cities.loc[1].NBA = ['Lakers', 'Clippers']
cities = cities.explode('NBA').reset_index(drop=True)
cities.rename(columns={'NBA': 'team'}, inplace=True)

df = pd.merge(nba_df, cities, on='team')
columns = ['W', 'L', 'win_rate', 'population']
for col in columns:
    df[col] = pd.to_numeric(df[col])

def nba_correlation():
    global data_nba
    data_nba = df.groupby('area').mean()
    
    population_by_region = [i for i in data_nba.population] # pass in metropolitan area population from cities
    win_loss_by_region = [i for i in data_nba.win_rate] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

mlb_df = mlb_df[mlb_df.year == 2018]
mlb_df.rename(columns={'W-L%': 'win_rate'}, inplace=True)
mlb_df = mlb_df[['team', 'win_rate']]
mlb_df['team'].replace({
    'Boston Red Sox': 'Red Sox',
    'New York Yankees': 'Yankees',
    'Tampa Bay Rays': 'Rays',
    'Toronto Blue Jays': 'Blue Jays',
    'Baltimore Orioles': 'Orioles',
    'Cleveland Indians': 'Indians',
    'Minnesota Twins': 'Twins',
    'Detroit Tigers': 'Tigers',
    'Chicago White Sox': 'White Sox',
    'Kansas City Royals': 'Royals',
    'Houston Astros': 'Astros',
    'Oakland Athletics': 'Athletics',
    'Seattle Mariners': 'Mariners',
    'Los Angeles Angels': 'Angels',
    'Texas Rangers': 'Rangers',
    'Atlanta Braves': 'Braves',
    'Washington Nationals': 'Nationals',
    'Philadelphia Phillies': 'Phillies',
    'New York Mets': 'Mets',
    'Miami Marlins': 'Marlins',
    'Milwaukee Brewers': 'Brewers',
    'Chicago Cubs': 'Cubs',
    'St. Louis Cardinals': 'Cardinals',
    'Pittsburgh Pirates': 'Pirates',
    'Cincinnati Reds': 'Reds',
    'Los Angeles Dodgers': 'Dodgers',
    'Colorado Rockies': 'Rockies',
    'Arizona Diamondbacks': 'Diamondbacks',
    'San Francisco Giants': 'Giants',
    'San Diego Padres': 'Padres'}, inplace=True)

cities = cities.iloc[:, [0, 1, -3]]
cities = cities.replace({'—': np.nan, r'\[note\s\d+\]': ''}, regex=True).replace('', np.nan).dropna()
cities.rename(columns={cities.columns[1]: 'population', cities.columns[0]: 'area'}, inplace=True)
cities.loc[0].MLB = ['Yankees', 'Mets']
cities.loc[1].MLB = ['Dodgers', 'Angels']
cities.loc[2].MLB = ['Giants', 'Athletics']
cities.loc[3].MLB = ['Cubs', 'White Sox']
cities = cities.explode('MLB').reset_index(drop=True)
cities.rename(columns={'MLB': 'team'}, inplace=True)

df = pd.merge(mlb_df, cities, on='team')
for col in ['win_rate', 'population']:
    df[col] = pd.to_numeric(df[col])
    
    
def mlb_correlation(): 
    global data_mlb
    data_mlb = df.groupby('area').mean()
    
    population_by_region = [i for i in data_mlb.population] # pass in metropolitan area population from cities
    win_loss_by_region = [i for i in data_mlb.win_rate] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

nfl_df = nfl_df[nfl_df.year==2018]
nfl_df = nfl_df[nfl_df.columns[[-2, -3]]]
nfl_df = nfl_df[nfl_df.team.str.contains('FC') == False]
nfl_df.rename(columns={'W-L%': 'win_rate'}, inplace=True)
nfl_df.team = nfl_df.team.replace('[\*\+]', value='', regex=True)
nfl_df.team.replace({
        'New England Patriots': 'Patriots',
        'Miami Dolphins': 'Dolphins',
        'Buffalo Bills': 'Bills',
        'New York Jets': 'Jets',
        'Baltimore Ravens': 'Ravens',
        'Pittsburgh Steelers': 'Steelers',
        'Cleveland Browns': 'Browns',
        'Cincinnati Bengals': 'Bengals',
        'Houston Texans': 'Texans',
        'Indianapolis Colts': 'Colts',
        'Tennessee Titans': 'Titans',
        'Jacksonville Jaguars': 'Jaguars',
        'Kansas City Chiefs': 'Chiefs',
        'Los Angeles Chargers': 'Chargers',
        'Denver Broncos': 'Broncos',
        'Oakland Raiders': 'Raiders',
        'Dallas Cowboys': 'Cowboys',
        'Philadelphia Eagles': 'Eagles',
        'Washington Redskins': 'Redskins',
        'New York Giants': 'Giants',
        'Chicago Bears': 'Bears',
        'Minnesota Vikings': 'Vikings',
        'Green Bay Packers': 'Packers',
        'Detroit Lions': 'Lions',
        'New Orleans Saints': 'Saints',
        'Carolina Panthers': 'Panthers',
        'Atlanta Falcons': 'Falcons',
        'Tampa Bay Buccaneers': 'Buccaneers',
        'Los Angeles Rams': 'Rams',
        'Seattle Seahawks': 'Seahawks',
        'San Francisco 49ers': '49ers',
        'Arizona Cardinals': 'Cardinals'}, inplace=True)

cities = cities.iloc[:, [0, 1, 2]]
cities.columns = ['area', 'population', 'team']
cities = cities.replace({'—': np.NaN, '\[note\s\d+\]': ''}, regex=True).replace('', np.NaN).dropna()
cities.loc[0].team = ['Giants', 'Jets']
cities.loc[1].team = ['Rams', 'Chargers']
cities.loc[2].team = ['49ers', 'Raiders']
cities = cities.explode('team')

df = pd.merge(nfl_df, cities, on='team')
for col in ['win_rate', 'population']:
    df[col] = pd.to_numeric(df[col])

def nfl_correlation(): 
    global data_nfl
    data_nfl = df.groupby('area').mean()
    
    population_by_region = [i for i in data_nfl.population] # pass in metropolitan area population from cities
    win_loss_by_region = [i for i in data_nfl.win_rate] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def sports_team_performance():
    data = [data_nfl, data_nba, data_nhl, data_mlb]
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    
    for i in range(4):
        data[i].rename(columns={'win_rate': f'win_rate_{sports[i]}'}, inplace=True)
    
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    for a in range(3):
        for b in range(a+1, 4):
            temp_df = pd.merge(data[a], data[b], on='area')
            temp_df.groupby('area').mean()
            p = stats.ttest_rel(temp_df[f'win_rate_{sports[a]}'], temp_df[f'win_rate_{sports[b]}'])[1]
            p_values.loc[sports[a], sports[b]] = p
            p_values.loc[sports[b], sports[a]] = p
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values